## Descripció

Anem a practicar i a familiaritzar-nos amb algoritmes de classificació.

## Nivell 1

**- Exercici 1:**
Crea almenys tres models de classificació diferents per intentar predir el millor possible l’endarreriment dels vols (ArrDelay) de DelayedFlights.csv. Considera si el vol ha arribat tard o no (ArrDelay > 0).


In [27]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from imblearn.metrics import geometric_mean_score

%matplotlib inline   

Baixem la base de dades i la netegem de la mateixa manera que el exercici anterior:

In [7]:
flights2008_df=pd.read_csv('C:\\Users\\march\\000 Treball Ju`yter 000\\Sprints_IT Academy_Data Science\\Sprint_9\\DelayedFlights.csv', )

In [8]:
flights2008_df.drop(['Year'],axis=1,inplace=True) #eliminem l'any perquè tots els vols són del mateix any!!
flights2008_df['TailNum'].fillna(value='Unknown',inplace=True) #canviem vols sens número de cua per unknown

In [9]:
flights2008_df.drop(['Unnamed: 0'],axis=1,inplace=True)

In [10]:
flights2008_df.ArrDelay=flights2008_df.ArrDelay.map(lambda x: 0 if x<=0 else 1)
flights2008_df.groupby('ArrDelay').count()

,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,...,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
ArrDelay,,,,,,,,,,,,,,,,,,,,,
0,204956,204956,204956,204956,204956,204956,204956,204956,204956,204956,...,204956,204956,204956,204956,204956,0,0,0,0,0
1,1731802,1731802,1731802,1731802,1731802,1724692,1731802,1731802,1731802,1731802,...,1724692,1731347,1731802,1731802,1731802,1247488,1247488,1247488,1247488,1247488


In [11]:
flights2008_df.ArrDelay.value_counts(normalize=True)

1    0.894176
0    0.105824
Name: ArrDelay, dtype: float64

In [12]:
flights2008_df.isnull().sum()

Month                     0
DayofMonth                0
DayOfWeek                 0
DepTime                   0
CRSDepTime                0
ArrTime                7110
CRSArrTime                0
UniqueCarrier             0
FlightNum                 0
TailNum                   0
ActualElapsedTime      8387
CRSElapsedTime          198
AirTime                8387
ArrDelay                  0
DepDelay                  0
Origin                    0
Dest                      0
Distance                  0
TaxiIn                 7110
TaxiOut                 455
Cancelled                 0
CancellationCode          0
Diverted                  0
CarrierDelay         689270
WeatherDelay         689270
NASDelay             689270
SecurityDelay        689270
LateAircraftDelay    689270
dtype: int64

Per les que el retras és nul, posem a 0 els retrassos diferents on no constava cap valor:

In [13]:

for i in ['CarrierDelay','WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay']:
    flights2008_df.loc[(flights2008_df.ArrDelay==0)&(flights2008_df[i].isnull()),i ]=0

flights2008_df.loc[(flights2008_df.ArrDelay==0)&(flights2008_df['ArrTime'].isnull()),'ArrTime']=flights2008_df.CRSArrTime

flights2008_df.isnull().sum()

Month                     0
DayofMonth                0
DayOfWeek                 0
DepTime                   0
CRSDepTime                0
ArrTime                7110
CRSArrTime                0
UniqueCarrier             0
FlightNum                 0
TailNum                   0
ActualElapsedTime      8387
CRSElapsedTime          198
AirTime                8387
ArrDelay                  0
DepDelay                  0
Origin                    0
Dest                      0
Distance                  0
TaxiIn                 7110
TaxiOut                 455
Cancelled                 0
CancellationCode          0
Diverted                  0
CarrierDelay         484314
WeatherDelay         484314
NASDelay             484314
SecurityDelay        484314
LateAircraftDelay    484314
dtype: int64

In [14]:
for i in [ 'ArrTime','ActualElapsedTime', 'CRSElapsedTime', 'AirTime', 'TaxiIn', 'TaxiOut',
       'CarrierDelay','WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay']:
       buit= flights2008_df[(flights2008_df.ArrDelay==0)&(flights2008_df[i].isnull())].count().Month
       print (f'{i} té {buit} instancies buides (nan) on no hi hagi retras.')

ArrTime té 0 instancies buides (nan) on no hi hagi retras.
ActualElapsedTime té 0 instancies buides (nan) on no hi hagi retras.
CRSElapsedTime té 0 instancies buides (nan) on no hi hagi retras.
AirTime té 0 instancies buides (nan) on no hi hagi retras.
TaxiIn té 0 instancies buides (nan) on no hi hagi retras.
TaxiOut té 0 instancies buides (nan) on no hi hagi retras.
CarrierDelay té 0 instancies buides (nan) on no hi hagi retras.
WeatherDelay té 0 instancies buides (nan) on no hi hagi retras.
NASDelay té 0 instancies buides (nan) on no hi hagi retras.
SecurityDelay té 0 instancies buides (nan) on no hi hagi retras.
LateAircraftDelay té 0 instancies buides (nan) on no hi hagi retras.


In [15]:
#Com que no hi han dates de vols no retrassat, es procedeix a eliminar els valors nuls:
index=list(flights2008_df[flights2008_df['LateAircraftDelay'].isnull()].index)#index de vols que tenen distancia nul
flights2008_df.drop(index=index, inplace=True)
flights2008_df.isnull().sum()

Month                0
DayofMonth           0
DayOfWeek            0
DepTime              0
CRSDepTime           0
ArrTime              0
CRSArrTime           0
UniqueCarrier        0
FlightNum            0
TailNum              0
ActualElapsedTime    0
CRSElapsedTime       0
AirTime              0
ArrDelay             0
DepDelay             0
Origin               0
Dest                 0
Distance             0
TaxiIn               0
TaxiOut              0
Cancelled            0
CancellationCode     0
Diverted             0
CarrierDelay         0
WeatherDelay         0
NASDelay             0
SecurityDelay        0
LateAircraftDelay    0
dtype: int64

Al netejar el dataframe no queden vols cancelats. Es poden borrar totes les columnes referents a la cancelació i desviació de vols.

In [16]:
flights2008_df.ArrDelay.value_counts(normalize=True) 

1    0.858889
0    0.141111
Name: ArrDelay, dtype: float64

Hi ha molts més vols amb retard que sense 85.9% vs 14.1% del total), s'haurà de treballar amb 2 classes desbalancejades. Cal veure quin model podem fer servir, per tal d'avaluar el model també és molt important determinar quin és el paràmetre que ens permet comparar entre mètodes. A la bibliografia trobem que per a un problema de classificació entre classes, on li donem donem igual valor a les dues i la classe majoritària representa >80% del total de valors, el paràmetre a seguir seria la G-Mean.

In [17]:
def check_classification_method(method_classifier,features,label, random): #si random=0 serà aletori!
    if random==0:
        random_state=None
    X_train, X_test, y_train, y_test = train_test_split(features, label, test_size=0.20, random_state=random)
    method=method_classifier
    method.fit(X_train, list(y_train))
    prediction= method.predict(X_test)
    return  prediction,y_test


In [26]:
# Retorna una lista con metrica de classificacion accuracy, matrix confusion, G-Mean, f1 score + informe 
# Generem DataFrame per a contenir després els resultats.
metrics_df = pd.DataFrame(columns = ['Model', 'Accuracy' , 'Confusion Matrix', 'G-Mean'])
def check_prediction(prediction,real):
    list=[]
    accuracy=accuracy_score(real,prediction)
    confusion=confusion_matrix(real, prediction)
    gmean_score=round(geometric_mean_score(real, prediction,average = 'binary'),3)
    f1=f1_score(real, prediction, average='binary', sample_weight=None, zero_division='warn')
    list_metrics=[accuracy, confusion,gmean_score,f1]
    report= classification_report(real, prediction)
    return list_metrics, report

In [19]:
# Eliminem les dades no numèriques del Dataframe:
df_small=flights2008_df.sample(n=50000,random_state=42)
objectiu=list(df_small.ArrDelay)     
df_small.drop('ArrDelay', axis=1, inplace=True)
categor_index=df_small.dtypes[df_small.dtypes!='object'].index
df_small=df_small[categor_index]
df_small.head()[:2]

,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,FlightNum,ActualElapsedTime,CRSElapsedTime,...,Distance,TaxiIn,TaxiOut,Cancelled,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
515702,3,12,3,1911.0,1847,2236.0,2211,552,145.0,144.0,...,936,6.0,19.0,0,0,24.0,0.0,1.0,0.0,0.0
466285,3,17,1,1033.0,935,1152.0,1059,322,79.0,84.0,...,397,9.0,10.0,0,0,0.0,0.0,0.0,0.0,53.0


K Neighbors classifier 

Support Vector Classification 



In [20]:
X_train, X_test, y_train, y_test = train_test_split(df_small, objectiu, test_size=0.20, random_state=42)
method=KNeighborsClassifier(n_neighbors=5)
method.fit(X_train, y_train)
prediction= method.predict(X_test)

In [ ]:
model_list=['KNN', 'Support Vector']

In [ ]:
for i in [KNeighborsClassifier(n_neighbors=5), SVC(gamma='auto')]:
    
    prediction, real = check_classification_method(i,df_small,objectiu,0)
    metric_list, inform = check_prediction (prediction, real)
    print (i)
    print(f'Accuracy {metric_list[0]}')
    print('confusion matrix', metric_list[1])
    print (inform)

()

In [21]:
prediction, real = check_classification_method(GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,max_depth=1, random_state=0),df_small,objectiu,0)
metric_list, inform  = check_prediction (prediction, real)
print ('Gradient Boosting Classification')
print(f'Accuracy {metric_list[0]}')
print('confusion matrix', metric_list[1])
print('G-Mean:', metric_list[2])
print (inform)


Gradient Boosting Classification
Accuracy 1.0
confusion matrix [[1433    0]
 [   0 8567]]
G-Mean: 1.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1433
           1       1.00      1.00      1.00      8567

    accuracy                           1.00     10000
   macro avg       1.00      1.00      1.00     10000
weighted avg       1.00      1.00      1.00     10000



In [25]:

names = [
    "Nearest Neighbors",
    "Linear SVM","Decision Tree",
    "Random Forest",
    "Neural Net",
    "AdaBoost",
    "Naive Bayes",
    "QDA",
]
classifiers = [
    KNeighborsClassifier(3),
    SVC(kernel="linear", C=0.025),
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    MLPClassifier(alpha=1, max_iter=1000),
    AdaBoostClassifier(),
    GaussianNB(),
    QuadraticDiscriminantAnalysis()]

j=0
resultats_llista=[]
Accuracy_list=[]
g_mean_list=[]
conf_matrix_list=[]

for i in classifiers:
    
    prediction, real = check_classification_method(i,df_small,objectiu,0)
    resultats_llista.append([names[j], prediction, real])
    metric_list, inform  = check_prediction (prediction, real)
    print ('* '*5,names[j],' *'*5)
    print(f'Accuracy {metric_list[0]}')
    Accuracy_list.append(metric_list[0])
    print('confusion matrix', metric_list[1])
    conf_matrix_list.append(metric_list[1])
    print('G-Mean:', metric_list[2])
    g_mean_list.append(metric_list[2])
    j+=1



* * * * *  Nearest Neighbors  * * * * *
Accuracy 0.8582
confusion matrix [[ 443  990]
 [ 428 8139]]
G-Mean: 0.542
* * * * *  Linear SVM  * * * * *
Accuracy 1.0
confusion matrix [[1433    0]
 [   0 8567]]
G-Mean: 1.0
* * * * *  Decision Tree  * * * * *
Accuracy 0.9904
confusion matrix [[1349   84]
 [  12 8555]]
G-Mean: 0.97
* * * * *  Random Forest  * * * * *
Accuracy 0.8625
confusion matrix [[  58 1375]
 [   0 8567]]
G-Mean: 0.201
* * * * *  Neural Net  * * * * *
Accuracy 0.9992
confusion matrix [[1426    7]
 [   1 8566]]
G-Mean: 0.997
* * * * *  AdaBoost  * * * * *
Accuracy 1.0
confusion matrix [[1433    0]
 [   0 8567]]
G-Mean: 1.0
* * * * *  Naive Bayes  * * * * *
Accuracy 1.0
confusion matrix [[1433    0]
 [   0 8567]]
G-Mean: 1.0
* * * * *  QDA  * * * * *
Accuracy 0.1433
confusion matrix [[1433    0]
 [8567    0]]
G-Mean: 0.0


C:\Users\march\anaconda3\envs\machineLearning37\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\march\anaconda3\envs\machineLearning37\lib\site-packages\sklearn\discriminant_analysis.py:903: RuntimeWarning: divide by zero encountered in power
  X2 = np.dot(Xm, R * (S ** (-0.5)))
C:\Users\march\anaconda3\envs\machineLearning37\lib\site-packages\sklearn\discriminant_analysis.py:903: RuntimeWarning: invalid value encountered in multiply
  X2 = np.dot(Xm, R * (S ** (-0.5)))
C:\Users\march\anaconda3\envs\machineLearning37\lib\site-packages\sklearn\discriminant_analysis.py:906: RuntimeWarning: divide by zero encountered in log
  u = np.asarray([np.sum(np.log(s)) for s in self.scalings_])
C:\Users\march\anaconda3\envs\machineLearning37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no pred

In [ ]:
names = [
    
    "Naive Bayes"
]
classifiers = [
    GaussianNB()]

j=0
resultats_llista3=[]
for i in classifiers:
    
    prediction, real = check_classification_method(i,df_small,objectiu,0)
    resultats_llista3.append([names[j], prediction, real])
    acc, conf, inform = check_prediction (prediction, real)
    print (names[j])
    print(f'Accuracy {acc}')
    print('confusion matrix', conf)
    print (inform)
    j+=1

In [ ]:
names = [
    "Naive Bayes",
    
]
classifiers = [
    GaussianNB()]

j=0
resultats_llista4=[]
for i in classifiers:
    
    prediction, real = check_classification_method(i,df_small,objectiu,0)
    resultats_llista4.append([names[j], prediction, real])
    acc, conf, inform = check_prediction (prediction, real)
    print (names[j])
    print(f'Accuracy {acc}')
    print('confusion matrix', conf)
    print (inform)
    j+=1

L'accuracy ha de ser >0.86 per a millorar la proporció real de la classe majoritària. S'observa que hi ha 2 casos:
  1. Els mètode donen una precissó que és anàlogo a la proporció de la classe majoritària. Indica que la classificació no és bona per a la classe minòritaria.
  2. Els mètodes donen una precissió i exactitud màxima perquè han trobat els parametres que correlacionen directaent amb el retard.

Fonts d'informació per a la selecció del model:
https://machinelearningmastery.com/framework-for-imbalanced-classification-projects/

Evaluació de model de classificació per a classes imbalancejades:
http://glemaitre.github.io/imbalanced-learn/generated/imblearn.metrics.geometric_mean_score.html

https://stackabuse.com/overview-of-classification-methods-in-python-with-scikit-learn/


![imagen](https://machinelearningmastery.com/wp-content/uploads/2019/12/How-to-Choose-a-Metric-for-Imbalanced-Classification-latest.png)

**- Exercici 2:**
Compara els models de classificació utilitzant la precisió (accuracy), una matriu de confiança i d’altres mètriques més avançades.

Està inclòs dintre l'exercici 1.

 **- Exercici 3:** Entrena’ls utilitzant els diferents paràmetres que admeten.


**- Exercici 4:** Compara el seu rendiment utilitzant l’aproximació traint/test o cross-validation.



## Nivell 2

**- Exercici 5:** Realitza algun procés d’enginyeria de variables per millorar-ne la predicció


## Nivell 3

**- Exercici 6:** No utilitzis la variable DepDelay a l’hora de fer prediccions
